In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import round,col, when, udf, format_number, abs, avg
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.types import DoubleType
import pp_events as pp

In [2]:
spark = SparkSession.builder.appName("xG5").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/26 02:50:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
events_shot = spark.read.csv('Datas/events_shot.csv', header=True, inferSchema=True)

In [4]:
# events_shot = pp.preprocessing(events,spark)

In [5]:
features = ['other_pp','from_fk','from_ti','from_corner','from_counter','from_gk','from_keeper','from_ko',
            'header','corner_type','fk_type','pk_type',
            'half_volley_technique','volley_technique','lob_technique','overhead_technique','backheel_technique','diving_h_technique',
            'distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure',
            'shot_aerial_won','shot_first_time','shot_one_on_one','shot_open_goal','shot_follows_dribble','players_inside_area']
target = ['goal']

In [6]:
input_size = len(features)
output_size = 2

In [17]:
from pyspark.ml.feature import VectorAssembler

# Define the assembler
feature_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")

# Transform the dataset
assembled_data = feature_assembler.transform(events_shot)
train_data, test_data = assembled_data.randomSplit([0.8, 0.2], seed=42)

In [8]:
from pyspark.ml.classification import LogisticRegression

# Define the logistic regression model
log_reg = LogisticRegression(featuresCol="features_vector", labelCol="goal", maxIter=10)
xg_model = log_reg.fit(train_data)
# Make predictions
predictions = xg_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

24/12/26 02:50:37 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


ROC-AUC: 0.8064468687686451


In [12]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol="features_vector", labelCol="goal", numTrees=100)
xg_model = rf.fit(train_data)
predictions = xg_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

24/12/26 02:53:47 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


ROC-AUC: 0.7813334077343806


In [13]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

# Define the Multilayer Perceptron classifier
layers = [input_size, 128, 64, 32, 16, output_size]

mlp = MultilayerPerceptronClassifier(featuresCol="features_vector", labelCol="goal", maxIter=100, layers=layers, blockSize=128, seed=1234)
xg_model = mlp.fit(train_data)

# Make predictions
predictions = xg_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

ROC-AUC: 0.8057324785364578


In [18]:
#### BEST MODEL SO FAR ####
from pyspark.ml.classification import GBTClassifier

# Define the Gradient-Boosted Tree classifier
gbt = GBTClassifier(featuresCol="features_vector", labelCol="goal", maxIter=50)
xg_model = gbt.fit(train_data)

# Make predictions
predictions = xg_model.transform(test_data)

evaluator = BinaryClassificationEvaluator(labelCol="goal", rawPredictionCol="rawPrediction")
roc_auc = evaluator.evaluate(predictions)
print(f"ROC-AUC: {roc_auc}")

ROC-AUC: 0.8135336689098985


#### For Gradient Boosted Trees

In [19]:
# Define a function to extract the probability of the goal (class 1)
def extract_goal_probability(probability):
    return float(probability[1])

# Register the function as a UDF
extract_goal_probability_udf = udf(extract_goal_probability, DoubleType())

# Create a new column with the goal probability
predictions_with_goal_prob = predictions.withColumn("goal_probability", extract_goal_probability_udf(col("probability")))

# Format the goal_probability to remove scientific notation
predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability", format_number(col("goal_probability"), 10))

# Add a new column showing the difference between shot_statsbomb_xg and goal_probability
predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference", format_number(col("shot_statsbomb_xg") - col("goal_probability"),10))

predictions_with_goal_prob = predictions_with_goal_prob.\
    withColumn("difference_abs", format_number(abs(col('difference')),10))

# Show the results
predictions_with_goal_prob.select("shot_statsbomb_xg", "goal_probability", "difference",'difference_abs').show(50,False)

+-----------------+----------------+-------------+--------------+
|shot_statsbomb_xg|goal_probability|difference   |difference_abs|
+-----------------+----------------+-------------+--------------+
|0.10166888       |0.1462401185    |-0.0445712385|0.0445712385  |
|0.026109         |0.0452431335    |-0.0191341335|0.0191341335  |
|0.064399794      |0.1391158998    |-0.0747161058|0.0747161058  |
|0.033228997      |0.0527047067    |-0.0194757097|0.0194757097  |
|0.14392917       |0.2454218257    |-0.1014926557|0.1014926557  |
|0.073996074      |0.0803258191    |-0.0063297451|0.0063297451  |
|0.7835           |0.8066456056    |-0.0231456056|0.0231456056  |
|0.10890295       |0.1079249595    |0.0009779905 |0.0009779905  |
|0.05416238       |0.0562240482    |-0.0020616682|0.0020616682  |
|0.07232883       |0.0699231436    |0.0024056864 |0.0024056864  |
|0.087863006      |0.0559036661    |0.0319593399 |0.0319593399  |
|0.121686585      |0.1459664686    |-0.0242798836|0.0242798836  |
|0.0638635

In [20]:
# average of the difference
avg_diff = predictions_with_goal_prob.select(avg(col("difference"))).collect()[0][0]
abs_avg_diff = predictions_with_goal_prob.select(avg(col("difference_abs"))).collect()[0][0]

print(f"Average difference: {avg_diff}")
print(f"Abs average difference: {abs_avg_diff}")

Average difference: -0.007900773161505548
Abs average difference: 0.03216420972981491


In [21]:
# round the goal_probability to 0 or 1, same for the shot_statsbomb_xg and compare with goal to check the accuracy of the model
# vs the accuracy of the shot_statsbomb_xg model
predictions_with_goal_prob = predictions_with_goal_prob.withColumn("goal_probability_rounded", round(col("goal_probability")))\
    .withColumn("shot_statsbomb_xg_rounded", round(col("shot_statsbomb_xg")))

# Check if the rounded probability matches the actual goal for your model
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_correct", when(col("goal") == col("goal_probability_rounded"), 1).otherwise(0))\
    .withColumn("statsbomb_correct", when(col("goal") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0))
# Calculate accuracy
accuracy_model = predictions_with_accuracy.selectExpr("avg(model_correct) as model_accuracy").first()["model_accuracy"]
accuracy_statsbomb = predictions_with_accuracy.selectExpr("avg(statsbomb_correct) as statsbomb_accuracy").first()["statsbomb_accuracy"]

print(f"Model Accuracy: {accuracy_model * 100:.2f}%")
print(f"StatsBomb xG Accuracy: {accuracy_statsbomb * 100:.2f}%")

Model Accuracy: 90.22%
StatsBomb xG Accuracy: 90.32%


In [22]:
# Testing matching of models, is out model as good as SB_xg?
predictions_with_accuracy = predictions_with_goal_prob.withColumn(
    "model_to_sbxg", when(col("goal_probability_rounded") == col("shot_statsbomb_xg_rounded"), 1).otherwise(0))

xg_accuracy_model = predictions_with_accuracy.selectExpr("avg(model_to_sbxg) as model_accuracy").first()["model_accuracy"]

print(f"Models matching percentage: {xg_accuracy_model * 100:.2f}%")

Models matching percentage: 98.57%


In [ ]:
# show each feature with its coefficient
coefficients = xg_model.stages[-1].coefficients
features_coefficients = list(zip(features, coefficients))

for feature, coefficient in features_coefficients:
    print(f"{feature}: {coefficient}")

In [23]:
train_predictions = xg_model.transform(train_data)
test_predictions = xg_model.transform(test_data)
# calculate accuracy for each model
train_accuracy = train_predictions.filter(train_predictions.goal == train_predictions.prediction).count() / train_predictions.count()
test_accuracy = test_predictions.filter(test_predictions.goal == test_predictions.prediction).count() / test_predictions.count()

print(f"Train accuracy: {train_accuracy * 100:.2f}%")
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

Train accuracy: 91.05%
Test accuracy: 90.22%
